# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
!pip install hvplot

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.9/161.9 kB 2.1 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
!cp "/content/drive/My Drive/python-api-challenge/WeatherPy/api_keys.py" .


In [8]:
!mkdir -p output_data
!cp "/content/drive/My Drive/python-api-challenge/WeatherPy/output_data/cities.csv" output_data/


In [9]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [10]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hradec kralove,50.2092,15.8328,-2.19,89,51,0.66,CZ,1738555820
1,1,callao,-12.0667,-77.1500,24.84,83,0,5.14,PE,1738555821
2,2,tura,25.5198,90.2201,23.54,33,0,1.50,IN,1738555823
3,3,yarmouth,41.7057,-70.2286,2.03,83,100,3.60,US,1738555650
4,4,puerto natales,-51.7236,-72.4875,6.50,91,72,3.23,CL,1738555676


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [11]:
# Filtered city_data_df
city_data_df['Humidity'] = city_data_df['Humidity'].astype(float)

city_data_df_filtered = city_data_df[['City', 'Lat', 'Lng', 'Humidity']]

coordinates = city_data_df_filtered[['Lat', 'Lng']]
humidity = city_data_df_filtered['Humidity']


In [13]:
!pip install geoviews holoviews cartopy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.3/547.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 55.7 MB/s eta 0:00:00


In [14]:
# Configure the map plot
map_plot_1 = city_data_df_filtered.hvplot.points(
    x='Lng', y='Lat',
    size='Humidity', color='City',
    geo = True,
    width=800, height=600,
    tiles='OSM'
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [15]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_city_data = city_data_df[
    (city_data_df['Max Temp'] > 21) &
    (city_data_df['Max Temp'] < 27) &
    (city_data_df['Cloudiness'] == 0) &
    (city_data_df['Wind Speed'] < 4.5)]

# Drop any rows with null values
filtered_city_data = filtered_city_data.dropna()

# Display sample data
filtered_city_data

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,tura,25.5198,90.2201,23.54,33.0,0,1.50,IN,1738555823
181,181,lazaro cardenas,17.9583,-102.2000,24.19,76.0,0,0.88,MX,1738556071
265,265,holualoa,19.6228,-155.9522,24.91,73.0,0,1.54,US,1738556189
305,305,palo santo,-25.5633,-59.3378,25.24,43.0,0,1.98,AR,1738556244
373,373,puntarenas,9.9763,-84.8384,25.34,70.0,0,1.68,CR,1738556340
382,382,la'ie,21.6477,-157.9253,22.84,67.0,0,1.79,US,1738556353
473,473,camana,-16.6228,-72.7111,22.20,84.0,0,2.32,PE,1738556483
507,507,juan de ayolas,-27.4000,-56.9000,24.90,50.0,0,1.96,PY,1738556529


### Step 3: Create a new DataFrame called `hotel_df`.

In [16]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_city_data[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
2,tura,IN,25.5198,90.2201,33.0,
181,lazaro cardenas,MX,17.9583,-102.2000,76.0,
265,holualoa,US,19.6228,-155.9522,73.0,
305,palo santo,AR,-25.5633,-59.3378,43.0,
373,puntarenas,CR,9.9763,-84.8384,70.0,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [17]:
# Set parameters to search for a hotel
radius = 10000
params = {"apiKey":geoapify_key,
          "radius":radius,
          "categories":"accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
tura - nearest hotel: Hotel Polo Orchid
lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
holualoa - nearest hotel: Kona Hotel
palo santo - nearest hotel: No hotel found
puntarenas - nearest hotel: Hotel Midey
la'ie - nearest hotel: No hotel found
camana - nearest hotel: Caja Arequipa
juan de ayolas - nearest hotel: Hotel Nacional de Turismo Ayolas


,City,Country,Lat,Lng,Humidity,Hotel Name
2,tura,IN,25.5198,90.2201,33.0,Hotel Polo Orchid
181,lazaro cardenas,MX,17.9583,-102.2000,76.0,Hotel Sol del Pacífico
265,holualoa,US,19.6228,-155.9522,73.0,Kona Hotel
305,palo santo,AR,-25.5633,-59.3378,43.0,No hotel found
373,puntarenas,CR,9.9763,-84.8384,70.0,Hotel Midey
382,la'ie,US,21.6477,-157.9253,67.0,No hotel found
473,camana,PE,-16.6228,-72.7111,84.0,Caja Arequipa
507,juan de ayolas,PY,-27.4000,-56.9000,50.0,Hotel Nacional de Turismo Ayolas


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [21]:
# Configure the map plot
map_plot = hotel_df.hvplot.points(
    x='Lng',
    y='Lat',
    geo=True,
    hover_cols=["City", "Country", "Hotel Name"],
    tiles='OSM',
    size=30,
    color='City',
    cmap='coolwarm'
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name)